# NoriNori（海苔）


你需要按如下方式为一些方格涂黑:
- 一个区域内有且仅有两个方格被涂黑.
- 每个涂黑的方格都是一个多米诺*的一部分. 多米诺可以跨过区域边界.
- 多米诺之间不能相连,但是对角线相接是可以的.

> 一个多米诺是相邻的两个方格组成的图形 (1x2 or 2x1).

-------



You need to shade some squares in the following way:
- In each region, exactly two squares are shaded black.
- Each shaded square is part of a domino*. A domino can span across region boundaries.
- Dominos cannot be connected to each other horizontally or vertically, but diagonal adjacency is allowed.

> A domino is a shape formed by two adjacent squares (1x2 or 2x1).

<https://www.puzzle-norinori.com>


In [1]:
from __future__ import print_function
from ortools.sat.python import cp_model as cp 

def norinori_solver(grid_ipt, X, Y):
    grid = grid_ipt.split(",")
    try:
        len(grid ) == X * Y
    except Exception:
        raise(f"检查输入棋盘的长度 length of grid != {X} * {Y}")
    
    model = cp.CpModel()
    cages = {}
    x = {}
    for i in range(Y):
        for j in range(X):
            x[i, j] = model.NewBoolVar(name = f"x[{i}, {j}]")
    
    for idx, letter in enumerate(grid):
        if letter not in cages:
            cages[letter] = [x[idx // X, idx % X]]
        else:
            cages[letter].append(x[idx // X, idx % X])

    def norinori_neightbours(pos_x, pos_y):
        
        res = []
        
        if pos_x + 1 < Y:
            res.append(x[pos_x + 1, pos_y])
        
        if pos_x - 1 >= 0:
            res.append(x[pos_x - 1, pos_y])
        
        if pos_y + 1 < X:
            res.append(x[pos_x, pos_y + 1])
        
        if pos_y - 1 >= 0:
            res.append(x[pos_x, pos_y - 1])
        
        return res
    
    for i in range(Y):
        for j in range(X):
            neighbours = norinori_neightbours(i, j)
            model.Add(sum(neighbours) == 1).OnlyEnforceIf(x[i, j])
    
    for _, value in cages.items():
        model.Add(sum(value) == 2)
    
    
    solver = cp.CpSolver()
    status = solver.Solve(model)
    
    if status == cp.OPTIMAL:
        for i in range(Y):
            for j in range(X):
                print(solver.Value(x[i, j]), end=" ")
            print()
        print()

        print("NumConflicts:", solver.NumConflicts())
        print("NumBranches:", solver.NumBranches())
        print("WallTime:", solver.WallTime())


# 规定一下输入的grid的形状

if __name__ == "__main__":
    grid = "1,1,1,2,2,2,3,3,1,1,7,7,3,4,4,1,7,8,5,5,4,4,4,8,4,4,4,6,4,8,4,6,6,6,6,6"
    norinori_solver(grid, 6, 6)
    
    # 这里用逗号隔开是因为真的有可能出现很多很多很多很多宫啊！
 

0 0 1 1 0 1 
1 1 0 0 0 1 
0 0 0 1 1 0 
1 1 0 0 0 1 
0 0 1 1 0 1 
1 1 0 0 0 0 

NumConflicts: 0
NumBranches: 0
WallTime: 0.00481



![](../assets/figures/Norinori1.png)

![](../assets/figures/Norinori.png)